In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *
import statistics
import numpy as np

config = machine_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)
null_model = NullModelRegression(config=config)

### Data Load and Preprocessing ###

In [3]:
raw_data = data_processor.load_data()

raw_data_2 = raw_data.drop(columns=['vendor_name', 'model_name', 'ERP'])

data_1 = data_processor.impute_missing_values(raw_data_2)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

## KNN Model ##

### Hyperparameter Tuning ###

In [4]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [6]:
features=['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX']

In [7]:
gamma = 1/(statistics.stdev(data_train[config['target_column']]))
gamma

0.0071906613411797165

#### Tuning k ####

In [6]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average MSE score with k=1: 18692.819047619043
Average MSE score with k=2: 17661.420809542098
Average MSE score with k=3: 18995.569669901513
Average MSE score with k=4: 19611.49917804948
Average MSE score with k=5: 20429.088741870484
Average MSE score with k=6: 21261.43363838865
Average MSE score with k=7: 21813.683190323467
Average MSE score with k=8: 22263.245673862275
Average MSE score with k=9: 23109.50510701653
Average MSE score with k=10: 24044.911264984657
Average MSE score with k=11: 24791.148339346157
Average MSE score with k=12: 25524.400740610767
Average MSE score with k=13: 26372.642408441294
Average MSE score with k=14: 27191.311096247235
Best k is 2 with the lowest average MSE score of 17661.420809542098


#### Tuning Gamma ####

In [7]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 17774.013336162112
Average MSE score with g=0.6: 17734.10464995688
Average MSE score with g=0.8: 17696.562832538002
Average MSE score with g=1.0: 17661.420809542098
Average MSE score with g=1.2: 17628.695996424074
Average MSE score with g=1.4: 17598.390491571034
Average MSE score with g=1.6: 17570.491502459376
Best g is 1.6 with the lowest average MSE score of 17570.491502459376


### Model Performance ###

In [8]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists for storing null model metrics
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    data_train_standardized = data_processor.standardize_data(train_set, train_set, features=features)
    data_test_standardized = data_processor.standardize_data(train_set,test_set,features=features)  

    # Train and evaluate 
    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']
    
    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)
    
    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)

    # Evaluate null model
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)
    
    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)


average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)

print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model r2 score: {average_null_model_r2}")
print(f"Average null model pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average r2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 19073.865381296106
Average null model MAE score: 86.17113664476773
Average null model r2 score: 0.0
Average null model pearson score: nan
Average MSE score for k=2, g=1.6: 7512.193936429954
Average MAE score for k=2, g=1.6: 34.97747282263274
Average r2 score for k=2, g=1.6: 0.6523904298418604
Average pearson score for k=2, g=1.6: 0.8250440267334402


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## Edited KNN ##

### Hyperparameter Tuning ###

In [8]:
epsilon = statistics.stdev(data_3[config['target_column']])
epsilon

160.83073308779512

#### Tuning Epsilon ####

In [10]:
hyperparameters = np.arange(0.1,2,0.2)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average score with e=0.1: 38778.07607077246
Average score with e=0.3: 30175.29096799274
Average score with e=0.5: 28840.71963897106
Average score with e=0.7: 30513.332061414454
Average score with e=0.9: 27420.17226756271
Average score with e=1.1: 26313.910440406624
Average score with e=1.3: 24855.84380880785
Average score with e=1.5: 24464.997863820077
Average score with e=1.7: 22967.24611659685
Average score with e=1.9: 22948.6021755046
Best e is 1.9 with the lowest average MSE score of 22948.6021755046


#### Tuning k ####

In [11]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average score with k=1: 24474.57619047619
Average score with k=2: 22948.6021755046
Average score with k=3: 23417.646879133114
Average score with k=4: 23214.49718321472
Average score with k=5: 23913.237952320113
Average score with k=6: 24213.783169836
Average score with k=7: 24775.255090475788
Average score with k=8: 25360.162611618678
Average score with k=9: 25766.91935639654
Best k is 2 with the lowest average MSE score of 22948.6021755046


#### Tuning gamma ####

In [12]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 23044.22232910942
Average MSE score with g=0.6: 23026.51392144658
Average MSE score with g=0.8: 23009.47285176583
Average MSE score with g=1.0: 22993.13512696233
Average MSE score with g=1.2: 22977.530115185957
Average MSE score with g=1.4: 22962.68046607314
Average MSE score with g=1.6: 22948.6021755046
Best g is 1.6 with the lowest average MSE score of 22948.6021755046


In [13]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists to store null model metrics for MAE, R^2, and Pearson
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    edited_train_set = knn_model.edited_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(edited_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    # Null model evaluation
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)

    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)

average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)

print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model R2 score: {average_null_model_r2}")
print(f"Average null model Pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average R2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average Pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 19073.865381296106
Average null model MAE score: 86.17113664476773
Average null model R2 score: 0.0
Average null model Pearson score: nan
Average MSE score for k=2, g=1.6: 7127.359746567536
Average MAE score for k=2, g=1.6: 34.31842830852905
Average R2 score for k=2, g=1.6: 0.6771279508406438
Average Pearson score for k=2, g=1.6: 0.8389989397901749


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## Condensed Knn ##

#### Tuning Epsilon ####

In [14]:
hyperparameters = np.arange(0.1,1.5,0.1)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average MSE score with e=0.1: 17447.56000454993
Average MSE score with e=0.2: 16896.84861439928
Average MSE score with e=0.3: 16704.28329553683
Average MSE score with e=0.4: 16750.02769167738
Average MSE score with e=0.5: 17084.35193071019
Average MSE score with e=0.6: 16968.831588224195
Average MSE score with e=0.7: 17029.359931273084
Average MSE score with e=0.8: 17397.454116927627
Average MSE score with e=0.9: 18898.697946914144
Average MSE score with e=1.0: 17904.628337784572
Average MSE score with e=1.1: 21869.996747574576
Average MSE score with e=1.2: 22378.684453243157
Average MSE score with e=1.3: 19910.341902811862
Average MSE score with e=1.4: 20142.66864165698
Best e is 0.3 with the lowest average MSE score of 16704.28329553683


#### Tuning k ####

In [15]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average MSE score with k=1: 16681.178571428572
Average MSE score with k=2: 16872.42236646374
Average MSE score with k=3: 17756.84179879653
Average MSE score with k=4: 18191.274619280284
Average MSE score with k=5: 19297.02914160776
Average MSE score with k=6: 20268.625101438483
Average MSE score with k=7: 21302.02019214633
Average MSE score with k=8: 23176.241596467455
Average MSE score with k=9: 23556.7728800896
Best k is 1 with the lowest average MSE score of 16681.178571428572


#### Tuning Gamma ####

In [16]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average MSE score with g=0.4: 16967.35238095238
Average MSE score with g=0.6: 16612.966666666667
Average MSE score with g=0.8: 17078.89285714286
Average MSE score with g=1.0: 16647.280952380952
Average MSE score with g=1.2: 16476.68095238095
Average MSE score with g=1.4: 16760.033333333333
Average MSE score with g=1.6: 16657.12857142857
Best g is 1.2 with the lowest average MSE score of 16476.68095238095


### Model Performance ###

In [17]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

# Lists to store null model metrics for MAE, R^2, and Pearson
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    condesed_train_set = knn_model.condensed_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(edited_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    # Null model evaluation
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_mse = Evaluation.mean_squared_error(test_set[config['target_column']], null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_set[config['target_column']], null_model_prediction)
    null_model_r2 = Evaluation.r2_coefficient(test_set[config['target_column']], null_model_prediction)
    null_model_pearson = Evaluation.pearsons_correlation(test_set[config['target_column']], null_model_prediction)

    null_model_scores.append(null_model_mse)
    null_model_mae_scores.append(null_model_mae)
    null_model_r2_scores.append(null_model_r2)
    null_model_pearson_scores.append(null_model_pearson)


average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_mse = sum(null_model_scores) / len(null_model_scores)
average_null_model_mae = sum(null_model_mae_scores) / len(null_model_mae_scores)
average_null_model_r2 = sum(null_model_r2_scores) / len(null_model_r2_scores)
average_null_model_pearson = sum(null_model_pearson_scores) / len(null_model_pearson_scores)


print(f"Average null model MSE score: {average_null_model_mse}")
print(f"Average null model MAE score: {average_null_model_mae}")
print(f"Average null model R2 score: {average_null_model_r2}")
print(f"Average null model Pearson score: {average_null_model_pearson}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average R2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average Pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average null model MSE score: 19073.865381296106
Average null model MAE score: 86.17113664476773
Average null model R2 score: 0.0
Average null model Pearson score: nan
Average MSE score for k=1, g=1.2: 19155.20615318417
Average MAE score for k=1, g=1.2: 74.39100688468159
Average R2 score for k=1, g=1.2: -0.4623748099263908
Average Pearson score for k=1, g=1.2: 0.717260614962077


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
